<a href="https://colab.research.google.com/github/NamanT98/Prompt-Generation/blob/main/LoRA_Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q datasets
!pip install -q peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 8.5 MB/s eta 0:00:00


In [ ]:
import torch
device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [ ]:
from transformers import AutoModelForCausalLM,AutoTokenizer
model_id='bigscience/bloomz-560m'
tokenizer=AutoTokenizer.from_pretrained(model_id)
model=AutoModelForCausalLM.from_pretrained(model_id).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/715 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

In [ ]:
def get_results(model,inputs,max_new_tokens=100):
    outputs=model.generate(
        # input_ids=inputs['input_ids'],
        # attention_mask=inputs['attention_mask'],
        **inputs,
        max_new_tokens=max_new_tokens,
        repetition_penalty=1.5,
        early_stopping=True,
        eos_token_id=tokenizer.eos_token_id
    )
    return outputs

In [ ]:
inputs=tokenizer('I want you to act as motivational coach',return_tensors='pt').to(device)
print(inputs)

{'input_ids': tensor([[   44,  4026,  1152,   427,  1769,   661, 22675,  3383, 67604]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}


In [ ]:
outputs=get_results(model,inputs)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:535: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


In [ ]:
print(tokenizer.batch_decode(outputs,skip_special_tokens=True))

['I want you to act as motivational coach']


# Preparing Dataset

In [ ]:
from datasets import load_dataset
data=load_dataset('fka/awesome-chatgpt-prompts')
data=data.map(lambda x: tokenizer(x['prompt']),batched=True)
train=data['train']
train=train.remove_columns('act')
print(train)

Generating train split:   0%|          | 0/153 [00:00<?, ? examples/s]

Map:   0%|          | 0/153 [00:00<?, ? examples/s]

Dataset({
    features: ['prompt', 'input_ids', 'attention_mask'],
    num_rows: 153
})


In [ ]:
import textwrap
print(textwrap.fill(train[0]['prompt']))

I want you to act as a linux terminal. I will type commands and you
will reply with what the terminal should show. I want you to only
reply with the terminal output inside one unique code block, and
nothing else. do not write explanations. do not type commands unless I
instruct you to do so. when i need to tell you something in english, i
will do so by putting text inside curly brackets {like this}. my first
command is pwd


# Fine tuning

In [ ]:
import peft
from peft import LoraConfig,PeftModel,get_peft_model

In [ ]:
config=LoraConfig(r=4,
                  lora_alpha=1,
                  target_modules=['query_key_value'],
                  lora_dropout=0.05,
                  bias='lora_only',
                  task_type='CAUSAL_LM')

In [ ]:
peft_model=get_peft_model(model,config)
print(peft_model.print_trainable_parameters())

trainable params: 466,944 || all params: 559,607,808 || trainable%: 0.0834
None


In [ ]:
import os
working_dir = './'

output_directory = os.path.join(working_dir, "peft_lab_outputs")

In [ ]:
from transformers import Trainer,TrainingArguments

training_args=TrainingArguments(output_dir=output_directory,
                                learning_rate=0.003,
                                auto_find_batch_size=True,
                                num_train_epochs=5)

In [ ]:
import transformers
trainer=Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=train,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

In [ ]:
trainer.train()

Step,Training Loss


Step,Training Loss


TrainOutput(global_step=195, training_loss=2.2242733686398237, metrics={'train_runtime': 85.2822, 'train_samples_per_second': 8.97, 'train_steps_per_second': 2.287, 'total_flos': 184524548284416.0, 'train_loss': 2.2242733686398237, 'epoch': 5.0})

In [ ]:
peft_model_path = os.path.join(output_directory, f"lora_model")
trainer.model.save_pretrained(peft_model_path)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
loaded_model = PeftModel.from_pretrained(model,
                                        peft_model_path,
                                        is_trainable=False)

In [ ]:
input_sentences = tokenizer("I want you to act as a motivational coach. ", return_tensors="pt").to(device)
foundational_outputs_sentence = get_results(loaded_model, input_sentences)

results=tokenizer.batch_decode(foundational_outputs_sentence, skip_special_tokens=True)

In [ ]:
print(textwrap.fill(results[0]))

I want you to act as a motivational coach.  I will provide some
information about the individual, and it is your job help them achieve
their goals through effective strategies designed specifically for
achieving those objectives. My first request is: "I need someone who
can advise me on how best approachable individuals should be in order
not only feel but also perform effectively at work" (person profile).
You may use any of these responses: "Provide advice regarding
leadership styles that are appropriate within an organization;
Provides insightful feedback when asked by employees during meetings
or functions related


In [ ]:
input_sentences = tokenizer("I want you to act as a ML professor. ", return_tensors="pt").to(device)
foundational_outputs_sentence = get_results(loaded_model, input_sentences)

results=tokenizer.batch_decode(foundational_outputs_sentence, skip_special_tokens=True)

In [ ]:
print(textwrap.fill(results[0]))

I want you to act as a ML professor.  I will provide some information
about the subject of my interest, and it is your job help develop
concepts into algorithms that can be used in real-world applications.
This could involve designing experiments or using different types from
artificial intelligence (AI) such as: machine learning tools like deep
neural networks for analyzing big data sets etc., creating models with
appropriate features and/or testing them on various
datasets/applications depending upon how they are useful enough – this
includes conducting research studies where results should ideally
appear within one day! My first
